In [160]:
import pandas as pd

In [161]:
#load the xls file without containing any links
df = pd.read_excel('testing_excel.xlsx', sheet_name='Greeting_Goodbyes')

In [162]:
df = df.replace(u'\xa0', u' ', regex=True)

In [163]:
#remove the " character from the dataframe
df = df.replace('"', '', regex=True)
df

,Greetings,Voicetype,Goodbyes
0,Hmm?,All types of NPCs,Until next time.
1,Need something?,All types of NPCs,Hmm hmm.
2,Yes?,All types of NPCs,"All right, then."
3,"Ours is to smile at your passing, friend.(F)",MaleArgonian,NaN
4,It fills us with rage. What harm does it bring...,MaleArgonian,NaN
...,...,...,...
155,What can I do for you?,FemaleYoungEager,NaN
156,"Oh, did you need something?",MaleYoungEager,NaN
157,"Oh, did you need something?",FemaleYoungEager,NaN
158,It's a fine day with you around.(F),FemaleYoungEager,NaN


In [164]:
#split the dataframe in two dataframes, one with the columns Greetings and Voicetype and the other with the columns Goodbye and Voicetype
Greetings = df[['Greetings', 'Voicetype']]

#Drop the rows where Greetings is NaN
Greetings = Greetings.dropna(subset=['Greetings'])

Goodbyes = df[['Goodbyes', 'Voicetype']]

#drop the rows where goodbyes is NaN
Goodbyes = Goodbyes.dropna(subset=['Goodbyes'])

In [165]:
Greetings

,Greetings,Voicetype
0,Hmm?,All types of NPCs
1,Need something?,All types of NPCs
2,Yes?,All types of NPCs
3,"Ours is to smile at your passing, friend.(F)",MaleArgonian
4,It fills us with rage. What harm does it bring...,MaleArgonian
...,...,...
155,What can I do for you?,FemaleYoungEager
156,"Oh, did you need something?",MaleYoungEager
157,"Oh, did you need something?",FemaleYoungEager
158,It's a fine day with you around.(F),FemaleYoungEager


In [166]:
Goodbyes

,Goodbyes,Voicetype
0,Until next time.,All types of NPCs
1,Hmm hmm.,All types of NPCs
2,"All right, then.",All types of NPCs
21,"I'll take my leave, then.",MaleCommander
22,"I'll take my leave, then.",MaleNordCommander
30,"I'll take my leave, then.",FemaleCommander
42,Now if you'll excuse me...,MaleCondescending
45,Now if you'll excuse me...,FemaleCondescending
60,Safe travels.,MaleDarkElf
63,Safe travels.,FemaleDarkElf


In [167]:
#create an new column in the greetings dataframe called Tone 
Greetings['Tone'] = 'Neutral'

# check the rows of the datafram to see if there is a (H) or a (F) in the rows and change the tone to Happy or Formal

for index, row in Greetings.iterrows():
    if '(H)' in row['Greetings']:
        Greetings.at[index, 'Tone'] = 'Hostile'
        #remove the (H) from the row
        Greetings.at[index, 'Greetings'] = Greetings.at[index, 'Greetings'].replace('(H)', '')
    elif '(F)' in row['Greetings']:
        Greetings.at[index, 'Tone'] = 'Friendly'
        #remove the (F) from the row
        Greetings.at[index, 'Greetings'] = Greetings.at[index, 'Greetings'].replace('(F)', '')
    

In [168]:
#remove the (H) and (F) from the rows of the Greetings column
Greetings['Greetings'] = Greetings['Greetings'].str.replace(r"\(H\)", "")

In [169]:
Greetings

,Greetings,Voicetype,Tone
0,Hmm?,All types of NPCs,Neutral
1,Need something?,All types of NPCs,Neutral
2,Yes?,All types of NPCs,Neutral
3,"Ours is to smile at your passing, friend.",MaleArgonian,Friendly
4,It fills us with rage. What harm does it bring...,MaleArgonian,Hostile
...,...,...,...
155,What can I do for you?,FemaleYoungEager,Neutral
156,"Oh, did you need something?",MaleYoungEager,Neutral
157,"Oh, did you need something?",FemaleYoungEager,Neutral
158,It's a fine day with you around.,FemaleYoungEager,Friendly


In [170]:
#check if there is a null in the voicetype column 
Greetings['Voicetype'].isnull().values.any()

False

In [171]:
import json

# create a dictionary with the structure above
Greetings_data = [
    {
    "Speaker": Greetings['Voicetype'][index],
    "Dialogue": Greetings['Greetings'][index],
    "Tone": Greetings['Tone'][index],
    "Event": "Greeting",
    "Condition": None
} for index, row in Greetings.iterrows()]


data = Greetings_data   

In [172]:
#do the same with the Goodbyes dataframe
Goodbyes_data = [
    {
    "Speaker": Goodbyes['Voicetype'][index],
    "Dialogue": Goodbyes['Goodbyes'][index],
    "Tone": None,
    "Event": "Goodbye",
    "Condition": None
} for index, row in Goodbyes.iterrows()]

data = data + Goodbyes_data

In [173]:
#import the second sheet of the excel file to a dataframe
vampire = pd.read_excel('testing_excel.xlsx', sheet_name='Vampire')
vampire = vampire.replace(u'\xa0', u' ', regex=True)

In [174]:
#duplicate the rows
vampire = vampire.copy()

vampire= pd.concat([vampire]*2, ignore_index=True)

# Replace the first 8 values of the 'Voicetype' column with the first value
vampire.loc[0:7, 'Dialogue'] = vampire['Dialogue'][0]

# Replace the next 8 values of the 'Voicetype' column starting from the 8th index with the value at the 8th index
vampire.loc[8:15, 'Dialogue'] = vampire['Dialogue'][9]

In [175]:
#remove the " character from the dataframe
vampire = vampire.replace('"', '', regex=True)

In [176]:
vampire

,Voicetype,Dialogue
0,MaleCommoner,Don't like those eyes you got. There's a bad h...
1,MaleCommonerAccented,Don't like those eyes you got. There's a bad h...
2,FemaleCommoner,Don't like those eyes you got. There's a bad h...
3,MaleEvenToned,Don't like those eyes you got. There's a bad h...
4,MaleEvenTonedAccented,Don't like those eyes you got. There's a bad h...
5,FemaleEvenToned,Don't like those eyes you got. There's a bad h...
6,MaleNord,Don't like those eyes you got. There's a bad h...
7,FemaleNord,Don't like those eyes you got. There's a bad h...
8,MaleCommoner,Your skin's as pale as the snow. You scared of...
9,MaleCommonerAccented,Your skin's as pale as the snow. You scared of...


In [177]:
# add the vampire dataframe to the json file
Vampire_data = [
    {
    "Speaker": vampire['Voicetype'][index],
    "Dialogue": vampire['Dialogue'][index],
    "Tone": None,
    "Event": "Player is a vampire",
    "Condition": None
} for index, row in vampire.iterrows()]

data = data + Vampire_data

In [178]:
werewolf = pd.read_excel('testing_excel.xlsx', sheet_name='Werewolf')
werewolf = werewolf.replace('"', '', regex=True)
werewolf = werewolf.replace(u'\xa0', u' ', regex=True)

In [179]:
werewolf

,Voicetype,Dialogue
0,Any guard,Werewolf!
1,Any guard,It's a werewolf! To arms! To arms!
2,Any guard,By the gods! It..it can't be!
3,MaleCommoner,Come on! You monster!
4,MaleCommonerAccented,Come on! You monster!
5,FemaleCommoner,Come on! You monster!
6,MaleEvenToned,Come on! You monster!
7,MaleEvenTonedAccented,Come on! You monster!
8,FemaleEvenToned,Come on! You monster!
9,MaleNord,Come on! You monster!


In [180]:
#add the werewolf dataframe to the json file
Werewolf_data = [
    {
    "Speaker": werewolf['Voicetype'][index],
    "Dialogue": werewolf['Dialogue'][index],
    "Tone": None,
    "Event": "Player is a werewolf",
    "Condition": "Player has transformed into a werewolf"
} for index, row in werewolf.iterrows()]

data = data + Werewolf_data

In [181]:
#import the Generic_Events sheet
generic_events = pd.read_excel('testing_excel.xlsx', sheet_name='Generic_Events')

In [182]:
generic_events = generic_events.replace('"', '', regex=True)
generic_events = generic_events.replace(u'\xa0', u' ', regex=True)

In [183]:
generic_events

,Event,Dialogue
0,Witnessing a theft,You dirty thief!
1,Witnessing a theft,"Stop, thief!"
2,You are trespassing,You're not supposed to be in here.
3,You are trespassing,"Last warning. Leave, now."
4,You are trespassing,"I'm not going to warn you again. Get out, or I..."
...,...,...
89,Witnessing a murder,Guess they deserved it...
90,Witnessing a murder,What's done is done.
91,Witnessing a murder,Well... that was an unpleasant bit of business.
92,Witnessing a murder,I didn't know you were so cold-blooded...


In [184]:
#add the generic_events dataframe to the json file

Generic_Events_data = [
    {
    "Speaker": "All types of NPCs",
    "Dialogue": generic_events['Dialogue'][index],
    "Tone": None,
    "Event": generic_events['Event'][index],    
    "Condition": None
} for index, row in generic_events.iterrows()]

data = data + Generic_Events_data

In [185]:
# import the Flames_Girls sheet

flames_gifts= pd.read_excel('testing_excel.xlsx', sheet_name='Flames_Gifts')

In [186]:
flames_gifts = flames_gifts.replace('"', '', regex=True)
flames_gifts = flames_gifts.replace(u'\xa0', u' ', regex=True)

In [187]:
#add the flames_gifts dataframe to the json file
Flames_Gifts_data = [
    {
    "Speaker": flames_gifts['Voicetype'][index],
    "Dialogue": flames_gifts['Dialogue'][index],
    "Tone": None,
    "Event": flames_gifts['Event'][index],
    "Condition": None
    } for index, row in flames_gifts.iterrows()]

data = data + Flames_Gifts_data

In [188]:
#import the fighting_over_item sheet

fighting_over_item = pd.read_excel('testing_excel.xlsx', sheet_name='fighting_over_item')
fighting_over_item = fighting_over_item.replace('"', '', regex=True)
fighting_over_item = fighting_over_item.replace(u'\xa0', u' ', regex=True)

In [189]:
fighting_over_item_data = [
    {
    "Speaker": fighting_over_item['Voicetype'][index],
    "Dialogue": fighting_over_item['Dialogue'][index],
    "Tone": None,
    "Event": fighting_over_item['Event'][index],
    "Speaker_Number": fighting_over_item['Line_of_dialogue'][index],
    "Condition": None
    } for index, row in fighting_over_item.iterrows()]

data = data + fighting_over_item_data

In [190]:
#read the sheet return&bumping&naked
return_bumping_naked = pd.read_excel('testing_excel.xlsx', sheet_name='return&bumping&naked')
return_bumping_naked = return_bumping_naked.replace('"', '', regex=True)
return_bumping_naked = return_bumping_naked.replace(u'\xa0', u' ', regex=True)

In [191]:
#add the return_bumping_naked dataframe to the json file
return_bumping_naked_data = [
    {
    "Speaker": return_bumping_naked['Voicetype'][index],
    "Dialogue": return_bumping_naked['Dialogue'][index],
    "Tone": None,
    "Event": return_bumping_naked['Event'][index],
    "Condition": None
    } for index, row in return_bumping_naked.iterrows()]

data = data + return_bumping_naked_data

In [192]:
# read the sheet Asking_armor
asking_armor = pd.read_excel('testing_excel.xlsx', sheet_name='Asking_armor')
asking_armor = asking_armor.replace('"', '', regex=True)
asking_armor = asking_armor.replace(u'\xa0', u' ', regex=True)

In [193]:
#add the asking_armor dataframe to the json file
asking_armor_data = [
    {
    "Speaker": asking_armor['Voicetype'][index],
    "Dialogue": asking_armor['Dialogue'][index],
    "Tone": None,
    "Event": asking_armor['Event'][index],
    "Condition": None,
    "Response_to": asking_armor['Response_to'][index]
    } for index, row in asking_armor.iterrows()]

data = data + asking_armor_data

In [194]:
#read the sheet Spells&robes
spells_robes = pd.read_excel('testing_excel.xlsx', sheet_name='Spells&robes')
spells_robes = spells_robes.replace('"', '', regex=True)
spells_robes = spells_robes.replace(u'\xa0', u' ', regex=True)
#remove the \u2019 character from the dataframe
spells_robes = spells_robes.replace(u'\u2019', "'", regex=True)
#remove the \u00a0 character from the dataframe
spells_robes = spells_robes.replace(u'\u00a0', ' ', regex=True)

In [195]:
#add the spells_robes dataframe to the json file
spells_robes_data = [
    {
    "Speaker": spells_robes['Voicetype'][index],
    "Dialogue": spells_robes['Dialogue'][index],
    "Tone": None,
    "Event": spells_robes['Event'][index],
    "Condition": None
    } for index, row in spells_robes.iterrows()]

data = data + spells_robes_data

In [196]:
#read the sheet Shouting&Corpses
shouting_corpses = pd.read_excel('testing_excel.xlsx', sheet_name='Shouting&Corpses')
shouting_corpses = shouting_corpses.replace('"', '', regex=True)
shouting_corpses = shouting_corpses.replace(u'\xa0', u' ', regex=True)
shouting_corpses = shouting_corpses.replace(u'\u2019', "'", regex=True)
shouting_corpses = shouting_corpses.replace(u'\u00a0', ' ', regex=True)

In [197]:
#add the shouting_corpses dataframe to the json file
shouting_corpses_data = [
    {
    "Speaker": shouting_corpses['Voicetype'][index],
    "Dialogue": shouting_corpses['Dialogue'][index],
    "Tone": None,
    "Event": shouting_corpses['Event'][index],
    "Condition": None
    } for index, row in shouting_corpses.iterrows()]

data = data + shouting_corpses_data

In [198]:
#add the sheet combat_dialogue to the json file
combat_dialogue = pd.read_excel('testing_excel.xlsx', sheet_name='combat_dialogue')
combat_dialogue = combat_dialogue.replace('"', '', regex=True)
combat_dialogue = combat_dialogue.replace(u'\xa0', u' ', regex=True)
combat_dialogue = combat_dialogue.replace(u'\u2019', "'", regex=True)
combat_dialogue = combat_dialogue.replace(u'\u00a0', ' ', regex=True)

In [199]:
combat_dialogue_data = [
    {
    "Speaker": "All types of NPCs",
    "Dialogue": combat_dialogue['Dialogue'][index],
    "Tone": None,
    "Event": combat_dialogue['Event'][index],
    "Condition": combat_dialogue['Condition'][index]
    } for index, row in combat_dialogue.iterrows()]

data = data + combat_dialogue_data

In [200]:
#add the sheet Persuation_check to the json file

persuasion_check = pd.read_excel('testing_excel.xlsx', sheet_name='Persuasion_check')
persuasion_check = persuasion_check.replace('"', '', regex=True)
persuasion_check = persuasion_check.replace(u'\xa0', u' ', regex=True)
persuasion_check = persuasion_check.replace(u'\u2019', "'", regex=True)
persuasion_check = persuasion_check.replace(u'\u00a0', ' ', regex=True)

In [201]:
persuasion_check_data = [
    {
    "Speaker": "All types of NPCs",
    "Dialogue": persuasion_check['Dialogue'][index],
    "Tone": None,
    "Event": persuasion_check['Event'][index],
    "Result": persuasion_check['Result'][index],
    } for index, row in persuasion_check.iterrows()]

data = data + persuasion_check_data

In [202]:
#add the sheet entering_buying to the json file
entering_buying = pd.read_excel('testing_excel.xlsx', sheet_name='entering_buying')
entering_buying = entering_buying.replace('"', '', regex=True)
entering_buying = entering_buying.replace(u'\xa0', u' ', regex=True)
entering_buying = entering_buying.replace(u'\u2019', "'", regex=True)
entering_buying = entering_buying.replace(u'\u00a0', ' ', regex=True)

In [203]:
entering_buying_data = [
    {
    "Speaker": entering_buying['Voicetype'][index],
    "Dialogue": entering_buying['Dialogue'][index],
    "Tone": None,
    "Event": entering_buying['Event'][index],
    "Condition": None
    } for index, row in entering_buying.iterrows()]

data = data + entering_buying_data

In [204]:
# add the sheet request_training to the json file

In [205]:
request_training = pd.read_excel('testing_excel.xlsx', sheet_name='requesting_training')
request_training = request_training.replace('"', '', regex=True)
request_training = request_training.replace(u'\xa0', u' ', regex=True)
request_training = request_training.replace(u'\u2019', "'", regex=True)
request_training = request_training.replace(u'\u00a0', ' ', regex=True)

In [206]:
request_training_data = [
    {
    "Speaker": request_training['Voicetype'][index],
    "Dialogue": request_training['Dialogue'][index],
    "Tone": None,
    "Event": request_training['Event'][index],
    "Condition": None
    } for index, row in request_training.iterrows()]

data = data + request_training_data

In [207]:
#add the sheet marriage to the json file
marriage = pd.read_excel('testing_excel.xlsx', sheet_name='marriage')
marriage = marriage.replace('"', '', regex=True)
marriage = marriage.replace(u'\xa0', u' ', regex=True)
marriage = marriage.replace(u'\u2019', "'", regex=True)
marriage = marriage.replace(u'\u00a0', ' ', regex=True)

In [208]:
marriage_data = [
    {
    "Speaker": marriage['Voicetype'][index],
    "Dialogue": marriage['Dialogue'][index],
    "Tone": None,
    "Event": marriage['Event'][index],
    "Condition": None,
    "Response_to": marriage['Response_to'][index]
    } for index, row in marriage.iterrows()]

In [209]:
data = data + marriage_data

In [210]:
#add the sheet wedding to the json file
wedding = pd.read_excel('testing_excel.xlsx', sheet_name='wedding')
wedding = wedding.replace('"', '', regex=True)
wedding = wedding.replace(u'\xa0', u' ', regex=True)
wedding = wedding.replace(u'\u2019', "'", regex=True)
wedding = wedding.replace(u'\u00a0', ' ', regex=True)

In [211]:
wedding_data = [
    {
    "Speaker": wedding['Voicetype'][index],
    "Dialogue": wedding['Dialogue'][index],
    "Tone": None,
    "Event": wedding['Event'][index],
    "Condition": wedding['Condition'][index],
    } for index, row in wedding.iterrows()]

data = data + wedding_data

In [212]:
# add the sheet orcs_khajiits to the json file

In [213]:
orcs_khajiits = pd.read_excel('testing_excel.xlsx', sheet_name='orcs_khajiits')
orcs_khajiits = orcs_khajiits.replace('"', '', regex=True)


In [214]:
orcs_khajiits

,Voicetype,Dialogue,Condition,Event
0,FemaleOrc,"What are you doing here, outlander? We don't d...",Player is not a blood-kin,Player enters an Orc Stronghold
1,FemaleOrc,This is our stronghold. Leave.,Player is not a blood-kin,Player enters an Orc Stronghold
2,FemaleOrc,Outsiders. Nothing but trouble.,Player is not a blood-kin,Player enters an Orc Stronghold
3,FemaleOrc,Malacath blesses us with this land. It's not f...,Player is not a blood-kin,Player enters an Orc Stronghold
4,FemaleOrc,This stronghold is for the Orcs. Leave. Now.,Player is not a blood-kin,Player enters an Orc Stronghold
...,...,...,...,...
73,FemaleKhajiit,"Khajiit are not welcome in the cities, so we m...",Khajiit is a guard,The Player greets the Khajiit Guard on the Kha...
74,FemaleKhajiit,"Some come to buy, but others come to steal. Kh...",Khajiit is a guard,The Player greets the Khajiit Guard on the Kha...
75,FemaleKhajiit,"Blessings of the moons upon you, traveler.",Khajiit is a guard,The Player greets the Khajiit Guard on the Kha...
76,FemaleKhajiit,"Khajiit is just a guard, and has no wares to s...",Khajiit is a guard,The Player greets the Khajiit Guard on the Kha...


In [215]:
orcs_khajiits.columns

Index(['Voicetype', 'Dialogue', 'Condition', 'Event'], dtype='object')

In [216]:
orcs_khajiits_data = [
    {
    "Speaker": orcs_khajiits['Voicetype'][index],
    "Dialogue": orcs_khajiits['Dialogue'][index],
    "Tone": None,
    "Event": orcs_khajiits['Event'][index],
    "Condition": orcs_khajiits['Condition'][index],
    } for index, row in orcs_khajiits.iterrows()]

data = data + orcs_khajiits_data

In [217]:
#add the sheet housecarl to the json file
housecarl = pd.read_excel('testing_excel.xlsx', sheet_name='Housecarl')
housecarl = housecarl.replace('"', '', regex=True)
housecarl = housecarl.replace(u'\xa0', u' ', regex=True)
housecarl = housecarl.replace(u'\u2019', "'", regex=True)
housecarl = housecarl.replace(u'\u00a0', ' ', regex=True)

In [218]:
housecarl_data = [
    {
    "Speaker": housecarl['Voicetype'][index],
    "Dialogue": housecarl['Dialogue'][index],
    "Tone": None,
    "Event": housecarl['Event'][index],
    "Condition": None,
    } for index, row in housecarl.iterrows()]

data = data + housecarl_data

In [219]:
# add the sheet Carriage to the json file
carriage = pd.read_excel('testing_excel.xlsx', sheet_name='Carriage')
carriage = carriage.replace('"', '', regex=True)
carriage = carriage.replace(u'\xa0', u' ', regex=True)
carriage = carriage.replace(u'\u2019', "'", regex=True)
carriage = carriage.replace(u'\u00a0', ' ', regex=True)

In [220]:
carriage_data = [
    {
    "Speaker": carriage['Voicetype'][index],
    "Dialogue": carriage['Dialogue'][index],
    "Tone": None,
    "Event": carriage['Event'][index],
    "Condition": None,
    } for index, row in carriage.iterrows()]

data = data + carriage_data

In [221]:
guards = pd.read_excel('testing_excel.xlsx', sheet_name='Guards')
guards = guards.replace('"', '', regex=True)
guards = guards.replace(u'\xa0', u' ', regex=True)
guards = guards.replace(u'\u2019', "'", regex=True)
guards = guards.replace(u'\u00a0', ' ', regex=True)

In [222]:
guards_data = [
    {
    "Speaker": "Guard",
    "Dialogue": guards['Dialogue'][index],
    "Tone": None,
    "Event": guards['Event'][index],
    "Condition": guards['Condition'][index],
    } for index, row in guards.iterrows()]

data = data + guards_data

In [223]:
guards_race = pd.read_excel('testing_excel.xlsx', sheet_name='Guards_race')
guards_race = guards_race.replace('"', '', regex=True)
guards_race = guards_race.replace(u'\xa0', u' ', regex=True)
guards_race = guards_race.replace(u'\u2019', "'", regex=True)
guards_race = guards_race.replace(u'\u00a0', ' ', regex=True)

In [224]:
guards_race

,Dialogue,Race,Condition,Event
0,"What is it, Argonian?",Argonian,NaN,Guard reacts to players race
1,"Stay out of trouble, Argonian.",Argonian,NaN,Guard reacts to players race
2,"Stay out of trouble, lizard.",Argonian,Guard is a Stormcloak,Guard reacts to players race
3,"Staying out of trouble, kinsman?",Nord,Guard is Nord,Guard reacts to players race
4,How can I help a brother Nord?,Nord,Guard is Nord,Guard reacts to players race
5,"Stay out of trouble, Nord.",Nord,Guard is not Nord,Guard reacts to players race
6,"Don't cross me, Elf.","Altmer, Bosmer, or Dunmer",Guard is Nord and a Stormcloak,Guard reacts to players race
7,"Stay out of trouble, Elf.","Altmer, Bosmer, or Dunmer",Guard is not a Stormcloak,Guard reacts to players race
8,"What is it, Elf?","Altmer, Bosmer, or Dunmer",Guard is not a Stormcloak,Guard reacts to players race
9,"Stay out of trouble, Khajiit.",Khajiit,NaN,Guard reacts to players race


In [225]:
guards_race_data = [
    {
        "Speaker": "Guard",
        "Dialogue": row['Dialogue'],
        "Tone": None,
        "Event": row['Event'],
        "Condition": row['Condition'],
        "Race": row['Race'],
    } for index, row in guards_race.iterrows()  # Ensure this matches the DataFrame you're indexing
]
data = data + guards_race_data

In [226]:
#add the sheet Guard_Equipment to the json file
guard_equipment = pd.read_excel('testing_excel.xlsx', sheet_name='Guard_Equipment')
guard_equipment = guard_equipment.replace('"', '', regex=True)
guard_equipment = guard_equipment.replace(u'\xa0', u' ', regex=True)
guard_equipment = guard_equipment.replace(u'\u2019', "'", regex=True)
guard_equipment = guard_equipment.replace(u'\u00a0', ' ', regex=True)

In [227]:
guard_equipment_data = [
    {
    "Speaker": "Guard",
    "Dialogue": guard_equipment['Dialogue'][index],
    "Tone": None,
    "Event": guard_equipment['Event'][index],
    "Condition": guard_equipment['Condition'][index],
    "Equipment": guard_equipment['Equipment'][index],
    } for index, row in guard_equipment.iterrows()]

data = data + guard_equipment_data

In [228]:
#add the sheet Guard_skill to the json file
guard_skill = pd.read_excel('testing_excel.xlsx', sheet_name='Guard_skill')
guard_skill = guard_skill.replace('"', '', regex=True)
guard_skill = guard_skill.replace(u'\xa0', u' ', regex=True)
guard_skill = guard_skill.replace(u'\u2019', "'", regex=True)
guard_skill = guard_skill.replace(u'\u00a0', ' ', regex=True)

In [229]:
guard_skill_data = [
    {
    "Speaker": "Guard",
    "Dialogue": guard_skill['Dialogue'][index],
    "Tone": None,
    "Event": guard_skill['Event'][index],
    "Condition": None,
    } for index, row in guard_skill.iterrows()]

data = data + guard_skill_data

In [230]:
# add the Stromcloak-imperial sheet to the json file
stromcloak_imperial = pd.read_excel('testing_excel.xlsx', sheet_name='Stromcloak-imperial')
stromcloak_imperial = stromcloak_imperial.replace('"', '', regex=True)
stromcloak_imperial = stromcloak_imperial.replace(u'\xa0', u' ', regex=True)
stromcloak_imperial = stromcloak_imperial.replace(u'\u2019', "'", regex=True)
stromcloak_imperial = stromcloak_imperial.replace(u'\u00a0', ' ', regex=True)

In [231]:

stromcloak_imperial_data = [
    {
    "Speaker": stromcloak_imperial['Guard'][index],
    "Dialogue": stromcloak_imperial['Dialogue'][index],
    "Tone": None,
    "Event": stromcloak_imperial['Event'][index],
    "Condition": stromcloak_imperial['Condition'][index],
    } for index, row in stromcloak_imperial.iterrows()]

data = data + stromcloak_imperial_data

In [232]:
#add the Guard_interactions sheet to the json file
guard_interactions = pd.read_excel('testing_excel.xlsx', sheet_name='Guard_interactions')
guard_interactions = guard_interactions.replace('"', '', regex=True)
guard_interactions = guard_interactions.replace(u'\xa0', u' ', regex=True)
guard_interactions = guard_interactions.replace(u'\u2019', "'", regex=True)
guard_interactions = guard_interactions.replace(u'\u00a0', ' ', regex=True)

In [233]:
#keep the rows until row 103
guard_interactions = guard_interactions[:104]
guard_interactions

,Dialogue,Location,Condition,Event
0,<Town>'s under my protection. You watch yourse...,Any,NaN,Player interacts with the guard in a specific ...
1,"You've come to Dawnstar at a strange time, fri...",Dawnstar,NaN,Player interacts with the guard in a specific ...
2,The people here are still plagued with nightma...,Dawnstar,NaN,Player interacts with the guard in a specific ...
3,"Dangerous post, Dawnstar. If the bandits don't...",Dawnstar,NaN,Player interacts with the guard in a specific ...
4,"Old lady Frida is a grouch, but she sure can w...",Dawnstar,NaN,Player interacts with the guard in a specific ...
...,...,...,...,...
99,"South of here, and west of the road, lies Cron...",Windhelm,NaN,Player interacts with the guard in a specific ...
100,"I've no problem with your kind, Orc. But that ...",Windhelm,the player is an Orc,Player interacts with the guard in a specific ...
101,I've no problem with Orcs. But that stronghold...,Windhelm,NaN,Player interacts with the guard in a specific ...
102,Across the river to the northeast lies Yngol B...,Windhelm,NaN,Player interacts with the guard in a specific ...


In [234]:
guard_interactions_data = [
    {
    "Speaker": "Guard",
    "Dialogue": guard_interactions['Dialogue'][index],
    "Tone": None,
    "Event": guard_interactions['Event'][index],
    "Condition": guard_interactions['Condition'][index],
    "Location": guard_interactions['Location'][index],
    } for index, row in guard_interactions.iterrows()]

data = data + guard_interactions_data

In [235]:
print(data)

[{'Speaker': 'All types of NPCs', 'Dialogue': 'Hmm?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': None}, {'Speaker': 'All types of NPCs', 'Dialogue': 'Need something?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': None}, {'Speaker': 'All types of NPCs', 'Dialogue': 'Yes?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': None}, {'Speaker': 'MaleArgonian', 'Dialogue': 'Ours is to smile at your passing, friend.', 'Tone': 'Friendly', 'Event': 'Greeting', 'Condition': None}, {'Speaker': 'MaleArgonian', 'Dialogue': 'It fills us with rage. What harm does it bring to us now?', 'Tone': 'Hostile', 'Event': 'Greeting', 'Condition': None}, {'Speaker': 'MaleArgonian', 'Dialogue': 'You have questions?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': None}, {'Speaker': 'FemaleArgonian', 'Dialogue': 'You have questions?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': None}, {'Speaker': 'MaleArgonian', 'Dialogue': 'Tidings.', 'Tone': 'Neutral', 'Event': 'Greeting', 'C

In [236]:
def convert_all_to_string(data):
    """
    Recursively convert all data in a dictionary or list to string.
    """
    if isinstance(data, dict):
        return {key: convert_all_to_string(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_all_to_string(item) for item in data]
    else:
        return str(data)

# Convert all data to string
string_data = convert_all_to_string(data)

In [237]:
# Write the shuffled data to a json file
with open('data.json', 'w') as f:
    f.write(json.dumps(string_data, indent=4))

In [238]:
print(string_data)

[{'Speaker': 'All types of NPCs', 'Dialogue': 'Hmm?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': 'None'}, {'Speaker': 'All types of NPCs', 'Dialogue': 'Need something?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': 'None'}, {'Speaker': 'All types of NPCs', 'Dialogue': 'Yes?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': 'None'}, {'Speaker': 'MaleArgonian', 'Dialogue': 'Ours is to smile at your passing, friend.', 'Tone': 'Friendly', 'Event': 'Greeting', 'Condition': 'None'}, {'Speaker': 'MaleArgonian', 'Dialogue': 'It fills us with rage. What harm does it bring to us now?', 'Tone': 'Hostile', 'Event': 'Greeting', 'Condition': 'None'}, {'Speaker': 'MaleArgonian', 'Dialogue': 'You have questions?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': 'None'}, {'Speaker': 'FemaleArgonian', 'Dialogue': 'You have questions?', 'Tone': 'Neutral', 'Event': 'Greeting', 'Condition': 'None'}, {'Speaker': 'MaleArgonian', 'Dialogue': 'Tidings.', 'Tone': 'Neutral', 'Event': 

In [239]:
# randomize the components of the json file
import random

random.shuffle(string_data)

# Write the shuffled data to a json file
with open('shuffled_data.json', 'w') as f:
    f.write(json.dumps(string_data, indent=4))